In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
from matplotlib.colors import ListedColormap
from scipy.spatial import distance
from scipy.stats import wasserstein_distance
from itertools import chain
from numpy import linalg as LA
import sklearn.preprocessing as preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
import math
import random
import copy
from sklearn.model_selection import cross_val_score, cross_validate, KFold
import lightgbm as lgb
import joblib
import shap
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, average_precision_score, confusion_matrix
import scipy.stats as ss


# Load real and synthetic data

In [ ]:
# load real data
real_data_df = pd.read_csv('/YOUR_LOCAL_PATH/original_training_data.csv') # note this is the training data for GAN training (not include the test data)
min_max_log = np.load('/YOUR_LOCAL_PATH/min_max_log.npy', allow_pickle=True).item()
# for key, min_max in min_max_log.items():
#     min_, max_ = min_max[0], min_max[1]
#     col_values = np.array(real_data_df[key])
#     real_data_df[key] = (1 - col_values)*max_ + col_values*min_
condition_columns = list(real_data_df.columns)[8:-4]
# real_data_df

In [ ]:
min_max_log

In [ ]:
print(np.sum(real_data_df['WHITE']))
print(np.sum(real_data_df['BLACK']))
print(np.sum(real_data_df['ASIAN']))
print(np.sum(real_data_df['HISPANIC']))
print(np.sum(real_data_df['UN']))
print(np.sum(real_data_df['OTHER']))
print('Num of y-positive record %d' % np.sum(real_data_df['DIE_1y']))
print('Num of y-negative record %d' % (len(real_data_df) - np.sum(real_data_df['DIE_1y'])))

In [ ]:
# load synthetic data
col_name_list = list(real_data_df.columns)
# model_id_list = [11,12,13,14,15]
# ckpt_id_list = [38,38,38,38,38]
model_id_list = [21,22,23,24,25]
ckpt_id_list = ['best','best','best','best','best']
# model_id_list = ['xxx']
# ckpt_id_list = [229]
syn_data_list = []
for model_id, ckpt_id in zip(model_id_list, ckpt_id_list):
    
    syn_data = np.load(f'/YOUR_LOCAL_PATH/GAN_training/syn/emrwgan_model_{model_id}_ckpt_{ckpt_id}.npy', allow_pickle=True)
    for i in range(len(col_name_list)-4):
        syn_data[:,i] = (syn_data[:,i] >= 0.5)*1.0
    syn_data_df = pd.DataFrame(syn_data, columns = col_name_list)
    positive_outcome_syn_data = syn_data_df[syn_data_df['DIE_1y'] == 1.0].values
    negative_outcome_syn_data = syn_data_df[syn_data_df['DIE_1y'] == 0.0].values
    syn_data_df = np.concatenate((positive_outcome_syn_data[:14243,:], negative_outcome_syn_data[:112279,:]), axis=0)
    syn_data_df = pd.DataFrame(syn_data_df, columns = col_name_list)
    
    syn_data_list.append(syn_data_df.values)
    
    
    

In [ ]:
syn_data_df

In [ ]:
print(np.sum(syn_data_df['WHITE']))
print(np.sum(syn_data_df['BLACK']))
print(np.sum(syn_data_df['ASIAN']))
print(np.sum(syn_data_df['HISPANIC']))
print(np.sum(syn_data_df['UN']))
print(np.sum(syn_data_df['OTHER']))

In [ ]:
utility_score = {}
privacy_score = {}

# Data utility evaluation 

## 1. Utility Metrics

### 1.a Dimension-wise distribution

In [ ]:
## Dimension-wise distribution: binary features

binary_d2d_average = {}
binary_d2d_sum = {}
binary_perc_real = []
FEATURE_COUNT = len(syn_data_df.columns) - 4
train_data = real_data_df.values
for c in range(0,FEATURE_COUNT):
    if c != 6:
        binary_perc_real.append(np.sum(train_data[:,c])/train_data.shape[0])
print(len(binary_perc_real))

fig, axs = plt.subplots(1, 5, figsize = (32,5.5))
plt.setp(axs, xticks=[0, 1.0, 1],
        yticks=[0, 1.0, 1])

# plt.xticks(fontsize=20)

sd2d_result = []
ad2d_result = []
for run in range(len(syn_data_list)):
    syn_data = syn_data_list[run]
    binary_perc_syn = []

    for c in range(0,FEATURE_COUNT):
        if c != 6:
            binary_perc_syn.append(np.sum(syn_data[:,c])/train_data.shape[0])
    # define categories for features
    cluster_list= np.array([0]*6 + [2] + [1]*len(condition_columns))
    # calculate sd2d for each synthetic data
    sd2d = np.sum(abs(np.array(binary_perc_syn) - np.array(binary_perc_real)))
    ad2d = np.sum(abs(np.array(binary_perc_syn) - np.array(binary_perc_real))) / FEATURE_COUNT * 1000.0 

    for index in [2,1,0]:
        if index == 0:
            marker = "s"
            color = 'black'
            label = 'Race'
        elif index == 1:
            marker = "o"
            color = 'pink'
            label = 'Diagnosis'
        else:
            marker = "d"
            color = 'green'
            label = 'Gender'

        axs[run].scatter(np.array(binary_perc_real)[cluster_list==index], np.array(binary_perc_syn)[cluster_list==index], color = color, s = 48, label=label,marker = marker,alpha=0.9)


    axs[run].set_xlabel("Real", fontsize = 20)
    axs[run].set_ylabel("Synthetic", fontsize = 20)
    axs[run].set_xlim(0, 1)
    axs[run].set_ylim(0, 1)
    axs[run].tick_params(labelsize=20)

    #axs[i,run-1].set_title(f'{syn_list[i]}_{run}', fontsize = 8)
    axs[run].plot([0, 1], [0, 1], ls="--", c=".1")
    axs[run].text(0.4, 0.10, f'APD = {ad2d:.2f}', fontsize = 20)
    sd2d_result.append(sd2d)
    ad2d_result.append(ad2d)

# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize=15, frameon=False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize=20, frameon=False)
plt.show()
# fig.savefig('./gan_com_5run_vumc_0630.eps',bbox_inches='tight',format='eps') 



In [ ]:
sd2d_result

In [ ]:
## Dimension-wise distribution: continuous features

print(np.min(list(real_data_df['AGE'])))
print(np.max(list(real_data_df['AGE'])))

print(np.min(list(real_data_df['BMI'])))
print(np.max(list(real_data_df['BMI'])))

print(np.min(list(real_data_df['DIASTOLIC'])))
print(np.max(list(real_data_df['DIASTOLIC'])))

print(np.min(list(real_data_df['SYSTOLIC'])))
print(np.max(list(real_data_df['SYSTOLIC'])))

continuous_w_d = {}
continuous_columns = ['AGE', 'BMI', 'DIASTOLIC', 'SYSTOLIC']

RUN_list = [1,2,3,4,5]

for column in continuous_columns:
    distances_to_real = []

    model_list_figure = ['Real']
    real_values = list(real_data_df[column])
    data_list = [real_values]
    color_list = ['maroon']
    for run in RUN_list:
        emrwgan_df = pd.DataFrame(syn_data_list[run-1], columns = list(real_data_df.columns))
        data_list.append(list(emrwgan_df[column]))
        color_list.append('blue')
        model_list_figure.append('EMR-WGAN')
    
    fig = plt.figure(figsize=(20, 3))
    
    print('\n %s' % column)
    for subplot_count in range(len(data_list)):
        
        if subplot_count > 0:
            distances_to_real.append(wasserstein_distance(real_values, data_list[subplot_count]))
        
        plt.subplot(1, 6, subplot_count+1)
        # fig = plt.figure(figsize = (3, 5))
        plt.hist(data_list[subplot_count], color = color_list[subplot_count])
        plt.xlabel("Value")
#         plt.ylim(0, 10000)
        plt.title(model_list_figure[subplot_count])
        plt.grid(color = 'blue', linestyle = '--', axis = 'y')

    plt.show()
    print("EMR-WGAN : mean: %.4f, std: %.4f" % (np.mean(distances_to_real), np.std(distances_to_real)))

    continuous_w_d[column] = distances_to_real
print(continuous_w_d)

In [ ]:
continuous_value = {}
norm_continuous_value = {}
    
for test, w_d in continuous_w_d.items():
    w_d = list(w_d)
    max_val = np.max(w_d)
    min_val = np.min(w_d)
    norm_w_d = (w_d - min_val) / (max_val-min_val) / 10
    continuous_value[test] = w_d
    norm_continuous_value[test] = list(norm_w_d)
    
continuous_value_df = pd.DataFrame.from_dict(continuous_value)
norm_continuous_value_df = pd.DataFrame.from_dict(norm_continuous_value)
sum_all_continuous_variables = np.sum(norm_continuous_value_df.values, axis = 1)
print(sum_all_continuous_variables)
norm_continuous_value_df

In [ ]:
combine_score_binary_cont = []
for run in RUN_list:
    combine_score_binary_cont.append((sd2d_result[run-1] + sum_all_continuous_variables[run-1])/(len(real_data_df.columns)-1) * 1000)

print('Dimension-wise distribution scores for all considered runs:', combine_score_binary_cont)
utility_score['Dimension-wise distribution'] = combine_score_binary_cont



### 1.b Column-wise correlation

In [ ]:
real_cor = np.corrcoef(np.transpose(train_data))
cwc = []
print("Column-wise correlation difference: \n")
for matrix in syn_data_list:
    np.random.seed(0)
    noise_matrix = (np.random.rand(len(matrix),len(matrix[0])) - 1) / 100000000    
    syn_cor = np.corrcoef(np.transpose(np.array(matrix) + noise_matrix))
    cwc.append(LA.norm(real_cor - syn_cor, 'fro') / len(real_data_df.columns) / len(real_data_df.columns) * 1000 * 1000)

print(cwc)
utility_score['Column-wise correlation'] = cwc


In [ ]:
syn_data_list[0][0]

### 1.c Latent cluster analysis

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
train_data_tmp = copy.deepcopy(train_data)
train_data_df = pd.DataFrame(train_data_tmp)
train_data_df.loc[:,[1456, 1457, 1458, 1459]] = min_max_scaler.fit_transform(train_data_df[[1456, 1457, 1458, 1459]].values)
NUM_C = 5 ## the number of clusters which has been optimized


In [ ]:
real_data_df

In [ ]:
max(real_data_df['AGE'])

In [ ]:
log_cluster_score_list = []
print("Latent deviation: \n")
for matrix in syn_data_list:
    matrix_df = pd.DataFrame(copy.deepcopy(matrix))
    matrix_df.loc[:,[1456, 1457, 1458, 1459]] = min_max_scaler.fit_transform(matrix_df[[1456, 1457, 1458, 1459]].values)
    mixed_data = np.concatenate((train_data_df.values,matrix_df.values), axis = 0)
    pca = PCA()
    pca_result = pca.fit_transform(mixed_data)
    sum_diag = np.sum(pca.explained_variance_ratio_)
    i = 1
    while  np.sum(pca.explained_variance_ratio_[:i]) < 0.8:
        i += 1
#     print(i, np.sum(pca.explained_variance_ratio_[:i])/np.sum(pca.explained_variance_ratio_))
    pca = PCA(n_components=i)
    pca_result = pca.fit_transform(mixed_data)
    
    kmeans_model = KMeans(n_clusters=NUM_C).fit(pca_result)
    cluster_aff = kmeans_model.labels_.tolist()
    real_syn_label = [1]*len(train_data_df.values) + [0]*len(matrix_df.values)
    
    cluster_score_sum = 0
    for label in range(NUM_C):
        indices_label = [i for i in range(len(cluster_aff)) if cluster_aff[i] == label]
        real_syn_for_label = [real_syn_label[i] for i in indices_label]
        ratio = np.sum(real_syn_for_label)/len(real_syn_for_label)
        cluster_score_sum += (ratio - 0.5)**2
    log_cluster_score_list.append(math.log2(cluster_score_sum/NUM_C))
print(log_cluster_score_list)
utility_score['Latent cluster analysis'] = log_cluster_score_list


### 1.d Medical concept abundance

In [ ]:
mca_train_data_df = real_data_df[condition_columns]
mca_train_data = np.sum(mca_train_data_df.values, axis=1)
n, bins, patches = plt.hist(x=mca_train_data, bins=20, color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')

mca_syn_data_list = []

for matrix in syn_data_list:
    syn_data_df = pd.DataFrame(data = matrix, columns = list(real_data_df.columns))
    mca_syn_data_df = syn_data_df[condition_columns]
    mca_syn_data = np.sum(mca_syn_data_df.values, axis=1).astype(int)
    count_in_bins = {}

    bin_counts = [0]*len(n) 
    for data_point in mca_syn_data:
        bin_number = data_point // (bins[1]-bins[0])
        if bin_number >= len(n):
            bin_counts[-1] += 1
        else:
            bin_counts[int(bin_number)] += 1

    mca_syn_data_list.append(np.sum(np.abs(np.array(bin_counts)-n))*0.5/len(mca_train_data))

print("Medical concept abundance distances:", mca_syn_data_list)
utility_score['Medical concept abundance'] = mca_syn_data_list


### 1.e Clinical knowledge violation

In [ ]:
disease_male_dict = {}
disease_female_dict = {}

gender_column = real_data_df['GENDER'].tolist()
for column in condition_columns:
    disease_column = real_data_df[column].tolist()
    patient_positive = [index for index in range(len(disease_column)) if disease_column[index] == 1]
    gender_positive_patient = [gender_column[index] for index in patient_positive]
    if np.sum(gender_positive_patient) == 0: # male
        disease_male_dict[column] = len(patient_positive)
    if np.sum(gender_positive_patient) == len(gender_positive_patient): # female
        disease_female_dict[column] = len(patient_positive)

    
sorted_disease_female_dict = sorted(disease_female_dict.items(), key=lambda kv: kv[1], reverse=True)
sorted_disease_male_dict = sorted(disease_male_dict.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
# male: 600: Hyperplasia of prostate
# male: 185: Cancer of prostate
# male: 605: Erectile dysfunction [ED]

# female: 649: Other conditions or status of the mother complicating pregnancy, childbirth, or the puerperium
# female: 655: Known or suspected fetal abnormality affecting management of mother
# female: 646: Other complications of pregnancy NEC

print('    Male codes: ')
for code in ['600', '185', '605']:
    disease_column = real_data_df[code].tolist()
    patient_positive = [index for index in range(len(disease_column)) if disease_column[index] == 1]
    gender_column = real_data_df['GENDER'].tolist()
    gender_positive_patient = [gender_column[index] for index in patient_positive]
    print('       ' + code + ': # total patients = ' + str(np.sum(disease_column)) + '; male percentage: ' + str((len(gender_positive_patient) - np.sum(gender_positive_patient))/len(gender_positive_patient)) + '; female percentage: ' + str(np.sum(gender_positive_patient)/len(gender_positive_patient)))

print('    Female codes: ')
for code in ['649', '655', '646']:
    disease_column = real_data_df[code].tolist()
    patient_positive = [index for index in range(len(disease_column)) if disease_column[index] == 1]
    gender_column = real_data_df['GENDER'].tolist()
    gender_positive_patient = [gender_column[index] for index in patient_positive]
    print('       ' + code + ': # total patients = ' + str(np.sum(disease_column)) + '; male percentage: ' + str((len(gender_positive_patient) - np.sum(gender_positive_patient))/len(gender_positive_patient)) + '; female percentage: ' + str(np.sum(gender_positive_patient)/len(gender_positive_patient)))


In [ ]:
run = 1
male_code_violation = []
female_code_violation = []
for matrix in syn_data_list:
    syn_data_df = pd.DataFrame(data = matrix, columns = list(real_data_df.columns))
    print('\nSynthetic data ' + str(run) + ':')
    print('    Male codes: ')
    male_viol_sum = 0
    for code in ['600', '185', '605']:
        disease_column = syn_data_df[code].tolist()
        patient_positive = [index for index in range(len(disease_column)) if disease_column[index] == 1]
        gender_column = syn_data_df['GENDER'].tolist()
        gender_positive_patient = [gender_column[index] for index in patient_positive]
        print('       ' + code + ': # total patients = ' + str(np.sum(disease_column)) + '; male percentage: ' + str((len(gender_positive_patient) - np.sum(gender_positive_patient))/len(gender_positive_patient)) + '; female percentage: ' + str(np.sum(gender_positive_patient)/len(gender_positive_patient)))
        male_viol_sum += np.sum(gender_positive_patient)/len(gender_positive_patient)
    male_code_violation.append(male_viol_sum/3)
    
    print('    Female codes: ')
    female_viol_sum = 0
    for code in ['649', '655', '646']:
        disease_column = syn_data_df[code].tolist()
        patient_positive = [index for index in range(len(disease_column)) if disease_column[index] == 1]
        gender_column = syn_data_df['GENDER'].tolist()
        gender_positive_patient = [gender_column[index] for index in patient_positive]
        print('       ' + code + ': # total patients = ' + str(np.sum(disease_column)) + '; male percentage: ' + str((len(gender_positive_patient) - np.sum(gender_positive_patient))/len(gender_positive_patient)) + '; female percentage: ' + str(np.sum(gender_positive_patient)/len(gender_positive_patient)))
        female_viol_sum += (len(gender_positive_patient) - np.sum(gender_positive_patient))/len(gender_positive_patient)
    female_code_violation.append(female_viol_sum/3)
    
    run += 1
    
utility_score['Clinical knowledge violation'] = np.array(male_code_violation) + np.array(female_code_violation)
print('Clinical knowledge violation: male code violation ', male_code_violation, 'female code violation ', female_code_violation)
print('Combined violation: ', np.array(male_code_violation) + np.array(female_code_violation))


### 1.f TSTR

In [ ]:
## Train on real training dataset and test on real testing data set

random.seed(a=2021, version=2)

RECALL_THRESHOLD = 0.6  # hold this to compute other threshold related metrics
LABEL_INDEX = 6  # label column index in the dataset
RACE_COL = [0,1,2,3,4,5]
COL_LIST = list(real_data_df.columns)[len(RACE_COL)+1:]
CAT_IDX_wo_RACE_label = list(np.linspace(0, len(COL_LIST) - 5, num=len(COL_LIST) - 4).astype(int))

## Load the dataset for model training. Here it is real data. 70%
train_real_data = pd.read_csv('/YOUR_LOCAL_PATH/original_training_data.csv').values.astype(np.float)
train_real_data_index = np.linspace(0,len(train_real_data)-1,len(train_real_data)).astype('int')
random.shuffle(train_real_data_index)
train_real_data = train_real_data[train_real_data_index]
train_real_label = train_real_data[:,LABEL_INDEX].astype(np.float)
train_real_data = np.delete(train_real_data, [LABEL_INDEX]+RACE_COL, axis = 1)
print('The number of records in training (real) data is:  %d' % len(train_real_data))
print('The number of features in training (real) data is:  %d' % len(train_real_data[0]))
print('Positive vs Negative ratio in training (real) data is: %f' % (np.sum(train_real_label)/len(train_real_label)))

## load the dataset for model evaluation. Here it is real data. 30%
eval_real_data = pd.read_csv('/YOUR_LOCAL_PATH/original_testing_data.csv').values.astype(np.float)
eval_real_label = eval_real_data[:,LABEL_INDEX].astype(np.float)
eval_real_data = np.delete(eval_real_data, [LABEL_INDEX]+RACE_COL, axis=1)
print('\nThe number of records in evaluation (real) data is: %d' % len(eval_real_data))
print('The number of features in training (real) data is: %d' % len(eval_real_data[0]))
print('Positive vs Negative ratio in evaluation (real) data is: %f' % (np.sum(eval_real_label)/len(eval_real_label)))

## model training
print('\n !!!!!!!!!!!!!!!!!!! training is starting !!!!!!!!!!!!!!!!!!! ')

gkf = KFold(n_splits=5, shuffle=True, random_state=0).split(X=train_real_data, y=train_real_label)


# ############
# # try some candidates here

# param_grid = {
#             'n_estimators': [500,1000],
#             'colsample_bytree': [0.8,0.9],
#             'max_depth': [15,20],
#             'num_leaves': [50,80],
#             'reg_alpha': [1.1, 1.3],
#             'min_split_gain': [0.3, 0.5],
#             'subsample': [0.6, 0.9],
#             'subsample_freq': [20, 40]
#             }
############

param_grid = {
            'n_estimators': [500],
            'colsample_bytree': [0.9],
            'max_depth': [15],
            'num_leaves': [50],
            'reg_alpha': [1.3],
            'min_split_gain': [0.3],
            'subsample': [0.9],
            'subsample_freq': [40]
            }

lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='binary', num_boost_round=2000, learning_rate=0.01, metric='auc',categorical_feature=CAT_IDX_wo_RACE_label, n_jobs = 20)
gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=gkf)
gbm = gsearch.fit(X=train_real_data, y=train_real_label)
print("Best parameters:\n")
print(gbm.best_params_)

y_scores = gbm.predict_proba(eval_real_data)
y_scores = y_scores[:,1]
trtr_7_3_auroc = roc_auc_score(y_score=y_scores, y_true=eval_real_label)
trtr_prauc = average_precision_score(eval_real_label, y_scores)
fpr, tpr, threshold_candidate = roc_curve(eval_real_label, y_scores)
thres_index = (tpr > RECALL_THRESHOLD).tolist().index(True)
thres = threshold_candidate[thres_index]
print("Threshold for fixing recall as %.2f is %.4f" % (RECALL_THRESHOLD, thres))
pred_y = np.array([(value>=thres)*1.0 for value in y_scores])

tn, fp, fn, tp = confusion_matrix(eval_real_label, pred_y).ravel()
trtr_ppv = tp / (tp + fp)
trtr_npv = tn / (tn + fn)
trtr_sens = tp / (tp + fn)
trtr_spes = tn / (tn + fp)
trtr_acc = (tn + tp) / (tn + fp + fn + tp)
print("      *** Test on real data AUROC: %.4f, PRAUC: %.4f, ACC: %.4f, PPV: %.4f, NPV: %.4f, Sensitivity: %.4f, Specificity: %.4f" % (
trtr_7_3_auroc, trtr_prauc, trtr_acc, trtr_ppv, trtr_npv, trtr_sens, trtr_spes))

explainer = shap.TreeExplainer(gbm.best_estimator_)
shap_df = pd.DataFrame(explainer.shap_values(eval_real_data)[1], columns = COL_LIST)
shap_df_abs = abs(shap_df)

feature_importance_mean = shap_df_abs.mean(axis=0).sort_values(ascending=False)
feature_importance_value = {}
for key, value in feature_importance_mean.items():
    if key in feature_importance_value.keys():
        feature_importance_value[key].append(value)
    else:
        feature_importance_value[key] = [value]

correlation_coeff_value = {}
for key in COL_LIST:
    corr_value = np.corrcoef(shap_df[key], eval_real_data[:, COL_LIST.index(key)])[1][0]
    if key in correlation_coeff_value.keys():
        correlation_coeff_value[key].append(corr_value)
    else:
        correlation_coeff_value[key] = [corr_value]
        
np.save('./result/feature_importance_TRTR.npy', feature_importance_value)

# np.save('./result/r_70_train_r_30_test_correl_coeff_value.npy', correlation_coeff_value)
# shap_df.to_csv('./result/r_70_train_r_30_test_feature_importance.csv')
# np.save('./result/r_70_train_r_30_test_y_estimate.npy', y_scores)
# np.save('./result/r_70_train_r_30_test_y_label.npy', eval_real_label)
# joblib.dump(gbm.best_estimator_, './result/r_70_train_r_30_test.pkl')

print('TRTR AUROC: %.4f' % trtr_7_3_auroc)
print('TRTR AUPRC: %.4f' % trtr_prauc)
print('TRTR ACCURACY: %.4f' % trtr_acc)
print('TRTR PPV: %.4f' % trtr_ppv)
print('TRTR NPV: %.4f' % trtr_npv)
print('TRTR RECALL: %.4f' % trtr_sens)
print('TRTR SPESIFICITY: %.4f' % trtr_spes)


In [ ]:
feature_importance_value


In [ ]:
## feature importance plot

viridis = cm.get_cmap('viridis', 256)
top = cm.get_cmap('Oranges_r', 128)
bottom = cm.get_cmap('Blues', 128)
newcolors = np.vstack((top(np.linspace(0, 1, 128)), bottom(np.linspace(0, 1, 128))))
newcmp = ListedColormap(newcolors, name='OrangeBlue')

def ABS_SHAP(feature_importance_value, correlation_coeff_value, top, colors):

    correlation_coeff_value_ = {}
    for key, value in correlation_coeff_value.items():
        correlation_coeff_value_[key] = np.mean(value)
    corr_df = pd.DataFrame.from_dict(correlation_coeff_value_, orient='index').fillna(0)
    corr_df.reset_index(inplace=True)
    corr_df.columns  = ['Variable','Corr']
    color_assigned = []
    for corr in list(corr_df['Corr']):
        if not np.isnan(corr):
            color_assigned.append(colors[int((corr+1)/2 * len(colors))])
        else:
            color_assigned.append([1,1,1,1])
    corr_df['Sign'] = color_assigned

    feature_importance_value_ = {}
    feature_importance_value_std = {}
    for key, value in feature_importance_value.items():
        feature_importance_value_[key] = np.mean(value)
        feature_importance_value_std[key] = np.std(value)

    feature_importance_std_df = pd.DataFrame.from_dict(feature_importance_value_std, orient='index').fillna(0)
    feature_importance_std_df.reset_index(inplace=True)
    feature_importance_std_df.columns = ['Variable','SHAP_std']

    feature_importance_df = pd.DataFrame.from_dict(feature_importance_value_, orient='index').fillna(0)
    feature_importance_df.reset_index(inplace=True)
    feature_importance_df.columns = ['Variable','SHAP_abs']

    feature_importance_df = feature_importance_df.merge(feature_importance_std_df, left_on='Variable',right_on='Variable',how='inner')

    k2 = feature_importance_df.merge(corr_df,left_on = 'Variable',right_on='Variable',how='inner')
    k2 = k2.sort_values(by='SHAP_abs',ascending = False).head(top).iloc[::-1]
    print(list(k2['Variable']))
    colorlist = k2['Sign']
    ax = k2.plot.barh(x='Variable',y='SHAP_abs',color = colorlist, figsize=(8,10),legend=False, xerr='SHAP_std', edgecolor='black')
    ax.set_xlabel("Feature contribution [SHAP Values].")
    ax.set_ylabel("Factors")
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.xaxis.grid(color='gray', linestyle='-.', linewidth=0.5)
#     plt.savefig('feature_imp_0.png', dpi=300) 
#     plt.savefig('feature_imp_0.png', dpi=300, bbox_inches='tight')

    fig, aaxx = plt.subplots(figsize=(0.2, 15))
    fig.subplots_adjust(bottom=0.5)
    cmap = matplotlib.colors.ListedColormap(colors)
    norm = matplotlib.colors.Normalize(vmin=-1, vmax=1)
    cb1 = matplotlib.colorbar.ColorbarBase(aaxx, cmap=cmap,
                                norm=norm, orientation='vertical')
#     plt.savefig('feature_imp_1.png', dpi=300, bbox_inches='tight')
    
    return k2

ABS_SHAP(feature_importance_value, correlation_coeff_value, 32, newcolors) 

In [ ]:
pd.read_csv('/YOUR_LOCAL_PATH/original_testing_data.csv')

In [ ]:
## Train on synthetic dataset and test on real testing data set

random.seed(a=2021, version=2)

RECALL_THRESHOLD = 0.6  # hold this to compute other threshold related metrics
LABEL_INDEX = 6  # label column index in the dataset
RACE_COL = [0,1,2,3,4,5]
COL_LIST = list(real_data_df.columns)[len(RACE_COL)+1:]
CAT_IDX_wo_RACE_label = list(np.linspace(0, len(COL_LIST) - 5, num=len(COL_LIST) - 4).astype(int))
## load the dataset for model evaluation. Here it is real data. 30%
eval_real_data = pd.read_csv('/YOUR_LOCAL_PATH/original_testing_data.csv').values.astype(np.float)
eval_real_label = eval_real_data[:,LABEL_INDEX].astype(np.float)
eval_real_data = np.delete(eval_real_data, RACE_COL + [LABEL_INDEX], axis=1)
print('\nThe number of records in evaluation (real) data is: %d' % len(eval_real_data))
print('The number of features in training (real) data is: %d' % len(eval_real_data[0]))
print('Positive vs Negative ratio in evaluation (real) data is: %f' % (np.sum(eval_real_label)/len(eval_real_label)))


pred_result = []
correlation_result = []

real_auroc_result = []
real_prauc_result = []
real_acc_result = []
real_ppv_result = []
real_npv_result = []
real_sens_result = []
real_spes_result = []

print('\n   !!!!!!!!!!!!!!!!!!! training is starting !!!!!!!!!!!!!!!!!!! ')

for run_ in range(len(model_id_list)):

    run = run_ + 1
    print('\n ######## Syn dataset %d ######## ' % run)
    syn_training_data = syn_data_list[run_]
    random.seed(a=2021, version=2)
    syn_training_data_index = np.linspace(0,len(syn_training_data)-1,len(syn_training_data)).astype('int')
    random.shuffle(syn_training_data_index)
    syn_training_data = syn_training_data[syn_training_data_index]
    syn_training_data_label = syn_training_data[:,LABEL_INDEX].astype(np.float)
    syn_training_data = np.delete(syn_training_data, [LABEL_INDEX]+RACE_COL, axis = 1)
    
    gkf = KFold(n_splits=5, shuffle=True, random_state=0).split(X=syn_training_data, y=syn_training_data_label)

    param_grid = {
            'n_estimators': [500],
            'colsample_bytree': [0.9],
            'max_depth': [15],
            'num_leaves': [50],
            'reg_alpha': [1.3],
            'min_split_gain': [0.3],
            'subsample': [0.9],
            'subsample_freq': [40]
            }

    lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='binary', num_boost_round=2000, learning_rate=0.01, metric='auc',categorical_feature=CAT_IDX_wo_RACE_label, n_jobs = 20)
    gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=gkf)
    gbm = gsearch.fit(X=syn_training_data, y=syn_training_data_label)
    print("Best parameters:\n")
    print(gbm.best_params_)

    y_scores = gbm.predict_proba(eval_real_data)
    y_scores = y_scores[:,1]
    tstr_auroc = roc_auc_score(y_score=y_scores, y_true=eval_real_label)
    tstr_prauc = average_precision_score(eval_real_label, y_scores)
    fpr, tpr, threshold_candidate = roc_curve(eval_real_label, y_scores)
    thres_index = (tpr > RECALL_THRESHOLD).tolist().index(True)
    thres = threshold_candidate[thres_index]
    print("Threshold for fixing recall as %.2f is %.4f" % (RECALL_THRESHOLD, thres))
    pred_y = np.array([(value>=thres)*1.0 for value in y_scores])

    tn, fp, fn, tp = confusion_matrix(eval_real_label, pred_y).ravel()
    tstr_ppv = tp / (tp + fp)
    tstr_npv = tn / (tn + fn)
    tstr_sens = tp / (tp + fn)
    tstr_spes = tn / (tn + fp)
    tstr_acc = (tn + tp) / (tn + fp + fn + tp)
    print("      *** Test on real data AUROC: %.4f, PRAUC: %.4f, ACC: %.4f, PPV: %.4f, NPV: %.4f, Sensitivity: %.4f, Specificity: %.4f" % (
    tstr_auroc, tstr_prauc, tstr_acc, tstr_ppv, tstr_npv, tstr_sens, tstr_spes))

    real_auroc_result.append(tstr_auroc)
    real_prauc_result.append(tstr_prauc)
    real_acc_result.append(tstr_acc)
    real_ppv_result.append(tstr_ppv)
    real_npv_result.append(tstr_npv)
    real_sens_result.append(tstr_sens)
    real_spes_result.append(tstr_spes)

    explainer = shap.TreeExplainer(gbm.best_estimator_)
#         shap_value_list.extend(explainer.shap_values(eval_real_data)[1])
    shap_df = pd.DataFrame(explainer.shap_values(eval_real_data)[1], columns = COL_LIST)
    shap_df_abs = abs(shap_df)

    feature_importance_mean = shap_df_abs.mean(axis=0).sort_values(ascending=False)
    feature_importance_value = {}
    for key, value in feature_importance_mean.items():
        if key in feature_importance_value.keys():
            feature_importance_value[key].append(value)
        else:
            feature_importance_value[key] = [value]
            
    np.save(f'./result/TSTR_feature_importance_run_{run}.npy', feature_importance_value)


print('TSTR AUROC: ', real_auroc_result)
utility_score['TSTR_auroc'] = real_auroc_result
        


In [ ]:
feature_importance_value

## 1.g TRTS

In [ ]:
## Train on real testing dataset and test on real training data set

random.seed(a=2021, version=2)

RECALL_THRESHOLD = 0.6  # hold this to compute other threshold related metrics
LABEL_INDEX = 6  # label column index in the dataset
RACE_COL = [0,1,2,3,4,5]
COL_LIST = list(real_data_df.columns)[len(RACE_COL)+1:]
CAT_IDX_wo_RACE_label = list(np.linspace(0, len(COL_LIST) - 5, num=len(COL_LIST) - 4).astype(int))

## Load the dataset for model training. Here it is real data. 70%
train_real_data = pd.read_csv('/YOUR_LOCAL_PATH/original_testing_data.csv').values.astype(np.float)
train_real_data_index = np.linspace(0,len(train_real_data)-1,len(train_real_data)).astype('int')
random.shuffle(train_real_data_index)
train_real_data = train_real_data[train_real_data_index]
train_real_label = train_real_data[:,LABEL_INDEX].astype(np.float)
train_real_data = np.delete(train_real_data, [LABEL_INDEX]+RACE_COL, axis = 1)
print('The number of records in training (real) data is:  %d' % len(train_real_data))
print('The number of features in training (real) data is:  %d' % len(train_real_data[0]))
print('Positive vs Negative ratio in training (real) data is: %f' % (np.sum(train_real_label)/len(train_real_label)))

## load the dataset for model evaluation. Here it is real data. 30%
eval_real_data = pd.read_csv('/YOUR_LOCAL_PATH/original_training_data.csv').values.astype(np.float)
eval_real_label = eval_real_data[:,LABEL_INDEX].astype(np.float)
eval_real_data = np.delete(eval_real_data, [LABEL_INDEX]+RACE_COL, axis=1)
print('\nThe number of records in evaluation (real) data is: %d' % len(eval_real_data))
print('The number of features in training (real) data is: %d' % len(eval_real_data[0]))
print('Positive vs Negative ratio in evaluation (real) data is: %f' % (np.sum(eval_real_label)/len(eval_real_label)))

## model training
print('\n !!!!!!!!!!!!!!!!!!! training is starting !!!!!!!!!!!!!!!!!!! ')

gkf = KFold(n_splits=5, shuffle=True, random_state=0).split(X=train_real_data, y=train_real_label)


# ############
# # try some candidates here

# param_grid = {
#     'n_estimators': [500, 1000],
#     'colsample_bytree': [0.7, 0.8],
#     'max_depth': [15, 25],
#     'num_leaves': [20, 50],
#     'reg_alpha': [1.1, 1.3],
# #     'reg_lambda': [1.1, 1.3],
#     'min_split_gain': [0.3, 0.5],
#     'subsample': [0.8, 0.9],
#     'subsample_freq': [20]
#     }
############

param_grid = {
            'n_estimators': [500],
            'colsample_bytree': [0.9],
            'max_depth': [15],
            'num_leaves': [50],
            'reg_alpha': [1.3],
            'min_split_gain': [0.3],
            'subsample': [0.9],
            'subsample_freq': [40]
            }

lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='binary', num_boost_round=2000, learning_rate=0.01, metric='auc',categorical_feature=CAT_IDX_wo_RACE_label, n_jobs = 20)
gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=gkf)
gbm = gsearch.fit(X=train_real_data, y=train_real_label)
print("Best parameters:\n")
print(gbm.best_params_)

y_scores = gbm.predict_proba(eval_real_data)
y_scores = y_scores[:,1]
trtr_auroc = roc_auc_score(y_score=y_scores, y_true=eval_real_label)
trtr_prauc = average_precision_score(eval_real_label, y_scores)
fpr, tpr, threshold_candidate = roc_curve(eval_real_label, y_scores)
thres_index = (tpr > RECALL_THRESHOLD).tolist().index(True)
thres = threshold_candidate[thres_index]
print("Threshold for fixing recall as %.2f is %.4f" % (RECALL_THRESHOLD, thres))
pred_y = np.array([(value>=thres)*1.0 for value in y_scores])

tn, fp, fn, tp = confusion_matrix(eval_real_label, pred_y).ravel()
trtr_ppv = tp / (tp + fp)
trtr_npv = tn / (tn + fn)
trtr_sens = tp / (tp + fn)
trtr_spes = tn / (tn + fp)
trtr_acc = (tn + tp) / (tn + fp + fn + tp)
print("      *** Test on real data AUROC: %.4f, PRAUC: %.4f, ACC: %.4f, PPV: %.4f, NPV: %.4f, Sensitivity: %.4f, Specificity: %.4f" % (
trtr_auroc, trtr_prauc, trtr_acc, trtr_ppv, trtr_npv, trtr_sens, trtr_spes))

explainer = shap.TreeExplainer(gbm.best_estimator_)
shap_df = pd.DataFrame(explainer.shap_values(eval_real_data)[1], columns = COL_LIST)
shap_df_abs = abs(shap_df)

feature_importance_mean = shap_df_abs.mean(axis=0).sort_values(ascending=False)
feature_importance_value = {}
for key, value in feature_importance_mean.items():
    if key in feature_importance_value.keys():
        feature_importance_value[key].append(value)
    else:
        feature_importance_value[key] = [value]

correlation_coeff_value = {}
for key in COL_LIST:
    corr_value = np.corrcoef(shap_df[key], eval_real_data[:, COL_LIST.index(key)])[1][0]
    if key in correlation_coeff_value.keys():
        correlation_coeff_value[key].append(corr_value)
    else:
        correlation_coeff_value[key] = [corr_value]

# np.save('./result/r_70_train_r_30_test_correl_coeff_value.npy', correlation_coeff_value)
# shap_df.to_csv('./result/r_70_train_r_30_test_feature_importance.csv')
# np.save('./result/r_70_train_r_30_test_y_estimate.npy', y_scores)
# np.save('./result/r_70_train_r_30_test_y_label.npy', eval_real_label)
# joblib.dump(gbm.best_estimator_, './result/r_70_train_r_30_test.pkl')

print('TRTR AUROC: %.4f' % trtr_auroc)
print('TRTR AUPRC: %.4f' % trtr_prauc)
print('TRTR ACCURACY: %.4f' % trtr_acc)
print('TRTR PPV: %.4f' % trtr_ppv)
print('TRTR NPV: %.4f' % trtr_npv)
print('TRTR RECALL: %.4f' % trtr_sens)
print('TRTR SPESIFICITY: %.4f' % trtr_spes)




In [ ]:
## Train on real testing dataset and test on synthetic training data set

random.seed(a=2021, version=2)

train_real_data = pd.read_csv('/YOUR_LOCAL_PATH/original_testing_data.csv').values.astype(np.float)
train_real_data_index = np.linspace(0,len(train_real_data)-1,len(train_real_data)).astype('int')
random.shuffle(train_real_data_index)
train_real_data = train_real_data[train_real_data_index]
train_real_label = train_real_data[:,LABEL_INDEX].astype(np.float)
train_real_data = np.delete(train_real_data, [LABEL_INDEX]+RACE_COL, axis = 1)
print('The number of records in training (real) data is:  %d' % len(train_real_data))
print('The number of features in training (real) data is:  %d' % len(train_real_data[0]))
print('Positive vs Negative ratio in training (real) data is: %f' % (np.sum(train_real_label)/len(train_real_label)))

gkf = KFold(n_splits=5, shuffle=True, random_state=0).split(X=train_real_data, y=train_real_label)

param_grid = {
            'n_estimators': [500],
            'colsample_bytree': [0.9],
            'max_depth': [15],
            'num_leaves': [50],
            'reg_alpha': [1.3],
            'min_split_gain': [0.3],
            'subsample': [0.9],
            'subsample_freq': [40]
            }

lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='binary', num_boost_round=2000, learning_rate=0.01, metric='auc',categorical_feature=CAT_IDX_wo_RACE_label, n_jobs = 20)

gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=gkf)
gbm = gsearch.fit(X=train_real_data, y=train_real_label)
print("Best parameters:\n")
print(gbm.best_params_)

print('\n   !!!!!!!!!!!!!!!!!!! evaluation is starting !!!!!!!!!!!!!!!!!!! ')

syn_auroc_result = []
syn_prauc_result = []
syn_acc_result = []
syn_ppv_result = []
syn_npv_result = []
syn_sens_result = []
syn_spes_result = []

for run_ in range(len(model_id_list)):

    run = run_ + 1
    print('\n ######## Syn dataset %d ######## ' % run)
    
    syn_data = syn_data_list[run_]
    random.seed(a=2021, version=2)
    syn_data_index = np.linspace(0,len(syn_data)-1,len(syn_data)).astype('int')
    random.shuffle(syn_data_index)
    syn_data = syn_data[syn_data_index]
    syn_label = syn_data[:,LABEL_INDEX].astype(np.float)
    syn_data = np.delete(syn_data, [LABEL_INDEX]+RACE_COL, axis = 1)

    y_scores = gbm.best_estimator_.predict_proba(syn_data)
    y_scores = y_scores[:,1]
    auroc = roc_auc_score(y_score=y_scores, y_true=syn_label)
    prauc = average_precision_score(syn_label, y_scores)
    fpr, tpr, threshold_candidate = roc_curve(syn_label, y_scores)
    thres_index = (tpr > RECALL_THRESHOLD).tolist().index(True)
    thres = threshold_candidate[thres_index]
    print("Threshold for fixing recall as %.2f is %.4f" % (RECALL_THRESHOLD, thres))
    pred_y = np.array([(value>=thres)*1.0 for value in y_scores])

    tn, fp, fn, tp = confusion_matrix(syn_label, pred_y).ravel()
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    sens = tp / (tp + fn)
    spes = tn / (tn + fp)
    acc = (tn + tp) / (tn + fp + fn + tp)
    print("      *** Test on syn data AUROC: %.4f, PRAUC: %.4f, ACC: %.4f, PPV: %.4f, NPV: %.4f, Sensitivity: %.4f, Specificity: %.4f" % (
    auroc, prauc, acc, ppv, npv, sens, spes))   

    syn_auroc_result.append(auroc)
    syn_prauc_result.append(prauc)
    syn_acc_result.append(acc)
    syn_ppv_result.append(ppv)
    syn_npv_result.append(npv)
    syn_sens_result.append(sens)
    syn_spes_result.append(spes)


print('TRTS AUROC: ', syn_auroc_result)
utility_score['TRTS_auroc'] = syn_auroc_result
    

## 1.h feature importance

In [ ]:
# Load the real dataset for model training.

random.seed(a=2021, version=2)

RECALL_THRESHOLD = 0.6  # hold this to compute other threshold related metrics
LABEL_INDEX = 6  # label column index in the dataset
RACE_COL = [0,1,2,3,4,5]
# CAT_IDX_wo_RACE_label = list(np.linspace(0, len(COL_LIST) - 5, num=len(COL_LIST) - 4).astype(int))


feature_importance_TRTR = np.load('./result/feature_importance_TRTR.npy', allow_pickle=True).item()
COL_LIST = list(real_data_df.columns)[len(RACE_COL)+1:]
top_feature_indices = [COL_LIST.index(name) for name in list(feature_importance_TRTR.keys())]
cat_or_not = [0,1,1,1,1,0,0] + [1]*18 + [0] + [1]*(len(COL_LIST)-26) # indicate mannually the binary features

train_real_data = pd.read_csv('/YOUR_LOCAL_PATH/original_training_data.csv').values.astype(np.float)
train_real_data_index = np.linspace(0,len(train_real_data)-1,len(train_real_data)).astype('int')
random.shuffle(train_real_data_index)
train_real_data = train_real_data[train_real_data_index]
train_real_label = train_real_data[:,LABEL_INDEX].astype(np.float)
train_real_data = np.delete(train_real_data, [LABEL_INDEX]+RACE_COL, axis = 1)
print('The number of records in training (real) data is:  %d' % len(train_real_data))
print('The number of features in training (real) data is:  %d' % len(train_real_data[0]))
print('Positive vs Negative ratio in training (real) data is: %f' % (np.sum(train_real_label)/len(train_real_label)))

## load the dataset for model evaluation. Here it is real data. 30%
eval_real_data = pd.read_csv('/YOUR_LOCAL_PATH/original_testing_data.csv').values.astype(np.float)
eval_real_label = eval_real_data[:,LABEL_INDEX].astype(np.float)
eval_real_data = np.delete(eval_real_data, [LABEL_INDEX]+RACE_COL, axis=1)
print('\nThe number of records in evaluation (real) data is: %d' % len(eval_real_data))
print('The number of features in training (real) data is: %d' % len(eval_real_data[0]))
print('Positive vs Negative ratio in evaluation (real) data is: %f' % (np.sum(eval_real_label)/len(eval_real_label)))

## model training
print('\n !!!!!!!!!!!!!!!!!!! training is starting !!!!!!!!!!!!!!!!!!! ')

test_candidates = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
auroc_performance = {}
auprc_performance = {}

for num_features in test_candidates:
    print('Testing top %d features: ' % num_features)

    selected_feature_indices = top_feature_indices[:num_features]
    selected_feature_cat_or_not = cat_or_not[:num_features]
    cat_indices = [i for i in range(len(selected_feature_cat_or_not)) if selected_feature_cat_or_not[i] == 1]
    
    gkf = KFold(n_splits=5, shuffle=True, random_state=0).split(X=train_real_data, y=train_real_label)

    param_grid = {
            'n_estimators': [500],
            'colsample_bytree': [0.9],
            'max_depth': [15],
            'num_leaves': [50],
            'reg_alpha': [1.3],
            'min_split_gain': [0.3],
            'subsample': [0.9],
            'subsample_freq': [40]
            }

    lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='binary', num_boost_round=2000, learning_rate=0.01, metric='auc',categorical_feature=cat_indices, n_jobs = 20)

    gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=gkf)
    gbm = gsearch.fit(X=train_real_data[:, selected_feature_indices], y=train_real_label)
    print("Best parameters:\n")
    print(gbm.best_params_)

    y_scores = gbm.predict_proba(eval_real_data[:, selected_feature_indices])
    y_scores = y_scores[:,1]
    auroc = roc_auc_score(y_score=y_scores, y_true=eval_real_label)
    prauc = average_precision_score(eval_real_label, y_scores)
    fpr, tpr, threshold_candidate = roc_curve(eval_real_label, y_scores)
    thres_index = (tpr > RECALL_THRESHOLD).tolist().index(True)
    thres = threshold_candidate[thres_index]
    print("      Threshold for fixing recall as %.2f is %.4f" % (RECALL_THRESHOLD, thres))
    pred_y = np.array([(value>=thres)*1.0 for value in y_scores])

    tn, fp, fn, tp = confusion_matrix(eval_real_label, pred_y).ravel()
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    sens = tp / (tp + fn)
    spes = tn / (tn + fp)
    acc = (tn + tp) / (tn + fp + fn + tp)
    print("      *** Test on real data AUROC: %.4f, PRAUC: %.4f, ACC: %.4f, PPV: %.4f, NPV: %.4f, Sensitivity: %.4f, Specificity: %.4f" % (
    auroc, prauc, acc, ppv, npv, sens, spes))
    
    auroc_performance[num_features] = auroc
    auprc_performance[num_features] = prauc
    
np.save('./result/auroc_performance_feature_imp.npy', auroc_performance)
np.save('./result/auprc_performance_feature_imp.npy', auprc_performance)
    

In [ ]:
auroc_performance

In [ ]:
font = {'family' : 'normal', 'size'  : 18}
matplotlib.rc('font', **font)
plt.figure(figsize=(7, 7)) 
plt.xlabel("Top features to include in training", fontsize=20)
plt.ylabel("AUROC", fontsize=20)
plt.ylim(0.8, 1.0)
df = pd.DataFrame(auroc_performance.items(), columns=['x_axis', 'y_axis'])
plt.plot('x_axis', 'y_axis', data=df, linestyle='-', marker='o')
plt.plot([0, 100], [trtr_7_3_auroc, trtr_7_3_auroc], linestyle=':')  ## the performance of training on 70% real data and testing on 30% testing data
plt.grid()
plt.show()

In [ ]:
for n, auroc in auroc_performance.items():
    print(n, auroc, auroc/trtr_7_3_auroc)

## select 95% performance as the threshold, thus 20 is the number of important features to achieve this.

In [ ]:
feature_imp_score = []
TRTR_top_features = set(list(feature_importance_TRTR.keys())[:20])
for run in range(len(syn_data_list)):
    run += 1
    feature_importance_value = np.load(f'./result/TSTR_feature_importance_run_{run}.npy', allow_pickle=True).item()
    syn_top_imp_features = []
    for key, imp in feature_importance_value.items():
        syn_top_imp_features.append(key)
        if len(syn_top_imp_features) == 20:
            syn_top_imp_features = set(syn_top_imp_features)
            intersection_imp_features = TRTR_top_features.intersection(syn_top_imp_features)
            feature_imp_score.append(len(intersection_imp_features)/len(TRTR_top_features))
            break
utility_score['Feature importance'] = feature_imp_score
feature_imp_score

In [ ]:
np.save('./result/utility_score.npy', utility_score)

## 2. Rank datasets and derive scores for selection

In [ ]:
utility_score

In [ ]:
dataset_ranks = {}
for metric, score in utility_score.items():
    if metric not in ['TSTR_auroc', 'TRTS_auroc', 'Feature importance']:
        ranks = ss.rankdata(score).astype(int)
        dataset_ranks[metric] = list(ranks)
    else:
        ranks = len(syn_data_list) + 1 - ss.rankdata(score).astype(int)
        dataset_ranks[metric] = list(ranks)
    print(metric, ranks)
np.save('./result/utility_score_dataset_ranks.npy', dataset_ranks)
dataset_ranks